In [102]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn.metrics


epochs = 70

In [103]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [104]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
#         char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

def createMatrices(sentences, word2Idx, label2Idx, case2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
#         charIndices = []
        labelIndices = []
        
        for word,label in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
#             charIdx = []
#             for x in char:
#                 charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
#             charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, labelIndices]) 
        
    return dataset

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def iterate_minibatches(dataset,batch_len): 
    start = 0
    for i in batch_len:
        tokens = []
        caseing = []
#         char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,c,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            caseing.append(c)
#             char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(caseing)

In [105]:
# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [106]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [107]:
trainSentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
devSentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
testSentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12705
3068
3160


In [108]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [109]:
# trainSentences = addCharInformatioin(train_sentences)
# devSentences = addCharInformatioin(dev_sentences)
# testSentences = addCharInformatioin(test_sentences)


In [110]:
print(trainSentences[0])

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [111]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,label in sentence:
            labelSet.add(label)
            words[token.lower()] = True
#             word[token]=True

In [112]:
print(labelSet)

{'I-MISC', 'I-LOC', 'B-LOC', 'B-PER', 'I-ORG', 'O', 'B-MISC', 'I-PER', 'B-ORG'}


In [113]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [114]:
print(label2Idx)

{'I-ORG': 4, 'O': 5, 'B-MISC': 6, 'B-ORG': 8, 'I-MISC': 0, 'I-LOC': 1, 'I-PER': 7, 'B-LOC': 2, 'B-PER': 3}


In [115]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [70]:
# string_words = ' '.join(words.keys())
# print(string_words)


In [116]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'allUpper': 2, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'initialUpper': 3, 'mainly_numeric': 5, 'allLower': 1, 'other': 4, 'numeric': 0}


In [72]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings_conll.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word.lower()]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [73]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[2]))

300


In [74]:
# characters=set()
# for word in words.keys():
#     for char in word:
#         characters.add(char)
# # print(characters)
# characters = set(characters)
# char2Idx={}
# for char in characters:
#     char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [75]:
print(trainSentences[0]) 

[['Ereignis', 'O'], ['und', 'O'], ['Erzählung', 'O'], ['oder', 'O'], [':', 'O']]


In [76]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx)
dev_set = createMatrices(devSentences,word2Idx, label2Idx, case2Idx)
test_set =createMatrices(testSentences, word2Idx, label2Idx, case2Idx)

In [77]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
# print((train_set[0][2]))

[[10296, 38277, 17527, 29285, 40756], [3, 1, 3, 1, 4], [5, 5, 5, 5, 5]]
5


In [78]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [79]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
output = concatenate([words, casing,])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_input (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    12806400    words_input[0][0]                
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 8)      64          casing_input[0][0]               
__________________________________________________________________________________________________
concatenat

In [94]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing = batch       
        model.train_on_batch([tokens, casing], labels)
        a.update(i)
        print(' ')

Epoch 0/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
10/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 1/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 2/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 3/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [===========

78/79 [============================>.] - ETA: 0s 
Epoch 5/70
 0/79 [..............................] - ETA: 0s 
 
 2/79 [..............................] - ETA: 2s 
 
 4/79 [>.............................] - ETA: 2s 
 
 6/79 [=>............................] - ETA: 2s 
 
 8/79 [==>...........................] - ETA: 2s 
 
11/79 [===>..........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 6/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
49/79 [=================>............] - ETA: 2s 


78/79 [============================>.] - ETA: 0s 
Epoch 7/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 8/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
 9/79 [==>...........................] - ETA: 2s 
 
65/79 [=======================>......] - ETA: 1s 


78/79 [============================>.] - ETA: 0s 
Epoch 9/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 1s 
 
 5/79 [>.............................] - ETA: 1s 
 
 7/79 [=>............................] - ETA: 2s 
 
 9/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 10/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 1s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 11/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 1s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
 9/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 12/70
 0/79 [............

78/79 [============================>.] - ETA: 0s 
Epoch 14/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 15/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 1s 
 
 7/79 [=>............................] - ETA: 2s 
 
37/79 [=============>................] - ETA: 3s 


...

78/79 [============================>.] - ETA: 0s 
Epoch 56/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 57/70
 0/79 [..............................] - ETA: 0s 
 
 2/79 [..............................] - ETA: 2s 
 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
 9/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 58/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 1s 
 
 
 6/79 [=>............................] - ETA: 1s 
 
 8/79 [==>...........................] - ETA: 1s 
 
12/79 [===>..........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 59/70
 0/79 [.........

78/79 [============================>.] - ETA: 0s 
Epoch 60/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 61/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 
 6/79 [=>............................] - ETA: 1s 
 
62/79 [======================>.......] - ETA: 2s 


78/79 [============================>.] - ETA: 0s 
Epoch 62/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 1s 
 
 5/79 [>.............................] - ETA: 1s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 63/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
77/79 [============================>.] - ETA: 0s 


78/79 [============================>.] - ETA: 0s 
Epoch 64/70
 0/79 [..............................] - ETA: 0s 
 
 2/79 [..............................] - ETA: 2s 
 
 4/79 [>.............................] - ETA: 2s 
 
 6/79 [=>............................] - ETA: 2s 
 
 8/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 65/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
 9/79 [==>...........................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 66/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 
Epoch 67/70
 0/79 [.............

78/79 [============================>.] - ETA: 0s 
Epoch 69/70
 0/79 [..............................] - ETA: 0s 
 
 
 3/79 [>.............................] - ETA: 2s 
 
 5/79 [>.............................] - ETA: 2s 
 
 7/79 [=>............................] - ETA: 2s 
 
78/79 [============================>.] - ETA: 0s 


In [ ]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

 769/3005 [======>.......................] - ETA: 16s

In [96]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='weighted' ))

2866/2867 [============================>.] - ETA: 0s0.5811135468197255
0.9766043179778378


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [97]:
id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))

51444


In [98]:
print(idx2Label)

{0: 'I-MISC', 1: 'I-LOC', 2: 'B-LOC', 3: 'B-PER', 4: 'I-ORG', 5: 'O', 6: 'B-MISC', 7: 'I-PER', 8: 'B-ORG'}


In [ ]:
id = 1
f = open('test_without_char.txt','wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

#         string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
        
        string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        print(pred_label)
        f.write(string)
        id+=1
    f.write('\n'.encode())
        